# 🤖 Entrenamiento de Modelos - Análisis de Sentimientos

## Objetivo
Entrenar múltiples modelos de Machine Learning para clasificar sentimientos en reseñas de películas.

**Modelos a entrenar:**
1. Naive Bayes (MultinomialNB)
2. Regresión Logística
3. Random Forest

**Vectorización:**
- CountVectorizer (Bag of Words)
- TfidfVectorizer (TF-IDF)

In [ ]:
# Importaciones
import pandas as pd
import numpy as np
import time
import sys
sys.path.append('..')

from src.train_models import (
    split_data,
    create_vectorizers,
    vectorize_data,
    train_naive_bayes,
    train_logistic_regression,
    train_random_forest,
    train_all_models,
    save_models
)

print('✅ Módulos importados')

In [ ]:
# Cargar datos procesados
df = pd.read_csv('../data/imdb_preprocessed.csv')
print(f'✅ Dataset cargado: {len(df)} reseñas')
print(f'Columnas: {list(df.columns)}')
print(f'\nPrimeras filas:')
df.head()

In [ ]:
# Dividir en Train/Test
X_train, X_test, y_train, y_test = split_data(df, test_size=0.2, random_state=42)

print('\n✅ Datos divididos correctamente')

## 📊 Vectorización de Texto

### ¿Qué es la vectorización?
Los modelos de ML no pueden trabajar directamente con texto. Necesitamos convertir las palabras en números.

### Técnicas de Vectorización:

**1. Bag of Words (CountVectorizer)**
- Cuenta la frecuencia de cada palabra
- Simple pero efectivo
- No considera la importancia relativa de las palabras

**2. TF-IDF (TfidfVectorizer)**
- Term Frequency - Inverse Document Frequency
- Penaliza palabras muy comunes
- Da más peso a palabras distintivas
- **Generalmente mejor para clasificación de texto**

In [ ]:
# Crear vectorizadores
count_vec, tfidf_vec = create_vectorizers()

print('\n📝 Probando ambos vectorizadores...')

In [ ]:
# Probar CountVectorizer
print('\n1️⃣ PROBANDO COUNT VECTORIZER (BAG OF WORDS)')
X_train_count, X_test_count, count_vec_fitted = vectorize_data(X_train, X_test, count_vec)

print(f'\nShape entrenamiento: {X_train_count.shape}')
print(f'Shape prueba: {X_test_count.shape}')
print(f'Tipo de datos: {type(X_train_count)}')

In [ ]:
# Probar TfidfVectorizer
print('\n2️⃣ PROBANDO TF-IDF VECTORIZER')
X_train_tfidf, X_test_tfidf, tfidf_vec_fitted = vectorize_data(X_train, X_test, tfidf_vec)

print(f'\nShape entrenamiento: {X_train_tfidf.shape}')
print(f'Shape prueba: {X_test_tfidf.shape}')
print(f'Tipo de datos: {type(X_train_tfidf)}')

## ✅ Decisión de Vectorizador

**Usaremos TF-IDF** por las siguientes razones:

1. ✅ Mejor manejo de palabras comunes
2. ✅ Da más importancia a palabras distintivas
3. ✅ Generalmente superior en tareas de clasificación de texto
4. ✅ Normalización automática de frecuencias

Ambos vectorizadores generan la misma dimensionalidad (5000 features con bigramas), pero TF-IDF produce representaciones más discriminativas.

In [ ]:
# Usaremos TF-IDF para el entrenamiento
X_train_vec = X_train_tfidf
X_test_vec = X_test_tfidf
vectorizer = tfidf_vec_fitted

print('✅ Vectorizador seleccionado: TF-IDF')
print(f'Features: {X_train_vec.shape[1]}')
print(f'Muestras entrenamiento: {X_train_vec.shape[0]}')

## 🚀 Entrenamiento de Modelos

### Modelo 1: Naive Bayes
- **Tipo**: Clasificador probabilístico
- **Ventaja**: Rápido, funciona bien con texto
- **Basado en**: Teorema de Bayes
- **Uso**: Baseline común para clasificación de texto

In [ ]:
# Entrenar Naive Bayes
start_time = time.time()
nb_model = train_naive_bayes(X_train_vec, y_train)
nb_time = time.time() - start_time

# Prueba rápida
nb_score = nb_model.score(X_test_vec, y_test)
print(f'\n📊 Accuracy en test: {nb_score:.4f}')
print(f'⏱️ Tiempo de entrenamiento: {nb_time:.2f}s')

### Modelo 2: Regresión Logística
- **Tipo**: Clasificador lineal
- **Ventaja**: Interpretable, robusto
- **Regularización**: L2 (Ridge)
- **Uso**: Excelente para alta dimensionalidad

In [ ]:
# Entrenar Regresión Logística
start_time = time.time()
lr_model = train_logistic_regression(X_train_vec, y_train)
lr_time = time.time() - start_time

# Prueba rápida
lr_score = lr_model.score(X_test_vec, y_test)
print(f'\n📊 Accuracy en test: {lr_score:.4f}')
print(f'⏱️ Tiempo de entrenamiento: {lr_time:.2f}s')

### Modelo 3: Random Forest
- **Tipo**: Ensemble de árboles de decisión
- **Ventaja**: Captura relaciones no lineales
- **Parámetros**: 100 árboles, profundidad máxima 50
- **Nota**: Más lento pero potencialmente más preciso

In [ ]:
# Entrenar Random Forest
start_time = time.time()
rf_model = train_random_forest(X_train_vec, y_train)
rf_time = time.time() - start_time

# Prueba rápida
rf_score = rf_model.score(X_test_vec, y_test)
print(f'\n📊 Accuracy en test: {rf_score:.4f}')
print(f'⏱️ Tiempo de entrenamiento: {rf_time:.2f}s ({rf_time/60:.2f} min)')

In [ ]:
# Crear diccionario de modelos
models = {
    'naive_bayes': nb_model,
    'logistic_regression': lr_model,
    'random_forest': rf_model
}

print('✅ Todos los modelos entrenados correctamente')

## 📊 Comparación de Tiempos de Entrenamiento

In [ ]:
# Tabla comparativa de tiempos
import matplotlib.pyplot as plt

times_df = pd.DataFrame({
    'Modelo': ['Naive Bayes', 'Logistic Regression', 'Random Forest'],
    'Tiempo (segundos)': [nb_time, lr_time, rf_time],
    'Accuracy': [nb_score, lr_score, rf_score]
})

print('\n⏱️ COMPARACIÓN DE TIEMPOS Y ACCURACY')
print('='*60)
print(times_df.to_string(index=False))

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de tiempos
axes[0].bar(times_df['Modelo'], times_df['Tiempo (segundos)'], 
           color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.8)
axes[0].set_title('Tiempo de Entrenamiento', fontweight='bold')
axes[0].set_ylabel('Segundos')
axes[0].tick_params(axis='x', rotation=45)

# Gráfico de accuracy
axes[1].bar(times_df['Modelo'], times_df['Accuracy'], 
           color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.8)
axes[1].set_title('Accuracy en Test', fontweight='bold')
axes[1].set_ylabel('Accuracy')
axes[1].set_ylim([0.7, 1.0])
axes[1].tick_params(axis='x', rotation=45)

# Agregar valores
for i, v in enumerate(times_df['Tiempo (segundos)']):
    axes[0].text(i, v, f'{v:.1f}s', ha='center', va='bottom')
for i, v in enumerate(times_df['Accuracy']):
    axes[1].text(i, v, f'{v:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Guardar modelos
save_models(models, vectorizer, output_dir='../models/')

print('\n✅ Modelos y vectorizador guardados exitosamente')

In [ ]:
# Verificar archivos guardados
import os

print('\n📁 ARCHIVOS GUARDADOS EN ../models/')
print('='*60)
model_files = os.listdir('../models/')
for file in model_files:
    file_path = os.path.join('../models/', file)
    size_mb = os.path.getsize(file_path) / (1024*1024)
    print(f'  ✓ {file:<30} {size_mb:>10.2f} MB')

## 🎯 Resumen del Entrenamiento

### ✅ Tareas Completadas:

1. **División de Datos**: 80% entrenamiento, 20% prueba
2. **Vectorización**: TF-IDF con 5000 features y bigramas
3. **Modelos Entrenados**: 3 modelos diferentes con distintas arquitecturas
4. **Persistencia**: Modelos y vectorizador guardados en disco

### 📊 Observaciones Iniciales:

- **Naive Bayes**: El más rápido, buen baseline
- **Logistic Regression**: Balance entre velocidad y accuracy
- **Random Forest**: Más lento pero con buen rendimiento

### 🔜 Próximos Pasos:

En el siguiente notebook realizaremos una **evaluación exhaustiva** de los modelos:
- Métricas detalladas (Precision, Recall, F1-Score)
- Matrices de confusión
- Curvas ROC
- Análisis de características importantes
- Selección del mejor modelo

---
**Siguiente notebook:** `04_evaluation.ipynb`